## Install packages and dependencies

In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

## Read diabetes data

In [2]:
# Specify the relative path to the CSV file
data_path = '../dataset/diabetes.csv'

In [3]:
# loading the diabetes dataset to a pandas DataFrame
diabetes = pd.read_csv(data_path)

In [4]:
# printing the first 5 rows of the dataset
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
# number of rows and columns in the dataset
diabetes.shape

(768, 9)

In [6]:
# getting the statistical measures of the data
diabetes.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [7]:
diabetes['Outcome'].value_counts()

Outcome
0    500
1    268
Name: count, dtype: int64

In [8]:
diabetes.groupby('Outcome').mean()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,109.980000,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000
1,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164


In [9]:
# separating the data and labels
X = diabetes.drop(columns = 'Outcome', axis=1)
Y = diabetes['Outcome']

In [10]:
print(X)

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  
0                       0.627   50  
1                       0.351   31  


In [11]:
print(Y)

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64


## Train Test Split

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, stratify=Y, random_state=2)
     
print(X.shape, X_train.shape, X_test.shape)

(768, 8) (614, 8) (154, 8)


## Training the Model

### Class weights for class imbalance

In [13]:
# Class counts
count_class_0 = 500
count_class_1 = 268

# Total number of samples
total_samples = count_class_0 + count_class_1

# Number of classes
num_classes = 2

# Calculate class weights
weight_class_0 = total_samples / (num_classes * count_class_0)
weight_class_1 = total_samples / (num_classes * count_class_1)

class_weights = {0: weight_class_0, 1: weight_class_1}
print(class_weights)

{0: 0.768, 1: 1.4328358208955223}


###  Initialize the RandomForestClassifier with the class weights

In [14]:
# Initialize RandomForestClassifier with calculated class weights and additional parameters
classifier= RandomForestClassifier(
        n_estimators=100,            # Number of trees in the forest
        max_depth=10,                # Maximum depth of the tree
        min_samples_split=2,         # Minimum number of samples required to split an internal node
        min_samples_leaf=1,          # Minimum number of samples required to be at a leaf node
        max_features='sqrt',         # Number of features to consider when looking for the best split
        random_state=42,             # Seed used by the random number generator
        class_weight=class_weights   # Class weights to handle imbalance
    )
# Print the classifier to verify
print(classifier)

RandomForestClassifier(class_weight={0: 0.768, 1: 1.4328358208955223},
                       max_depth=10, random_state=42)


### Train the model

In [15]:
#training the RandomForestClassifier 
classifier.fit(X_train, Y_train)

RandomForestClassifier(class_weight={0: 0.768, 1: 1.4328358208955223},
                       max_depth=10, random_state=42)

## Model Evaluation

### Accuracy Score

In [16]:
# accuracy score on the training data
X_train_prediction = classifier.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
     
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  1.0


In [17]:
# accuracy score on the test data
X_test_prediction = classifier.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
     
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.7727272727272727


## Making a Predictive System

In [18]:
input_data = (5,166,72,19,175,25.8,0.587,51)

# changing the input_data to numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the array as we are predicting for one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

prediction = classifier.predict(input_data_reshaped)
print(prediction)

if (prediction[0] == 0):
  print('The person is not diabetic')
else:
  print('The person is diabetic')

[1]
The person is diabetic


/opt/conda/envs/disease/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


## Saving the trained model

In [19]:
# Specify the path to save the model
model_path = '../models/diabetes_model.sav'

# Save the model
with open(model_path, 'wb') as file:
    pickle.dump(classifier, file)

In [20]:
# Load the saved model
with open(model_path, 'rb') as file:
    loaded_model = pickle.load(file)

# Verify the loaded model (optional)
print(loaded_model)

RandomForestClassifier(class_weight={0: 0.768, 1: 1.4328358208955223},
                       max_depth=10, random_state=42)


In [21]:
input_data = (5,166,72,19,175,25.8,0.587,51)

# changing the input_data to numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the array as we are predicting for one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

prediction = loaded_model.predict(input_data_reshaped)
print(prediction)

if (prediction[0] == 0):
  print('The person is not diabetic')
else:
  print('The person is diabetic')

[1]
The person is diabetic


/opt/conda/envs/disease/lib/python3.10/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [22]:
for column in X.columns:
  print(column)

Pregnancies
Glucose
BloodPressure
SkinThickness
Insulin
BMI
DiabetesPedigreeFunction
Age
